In [1]:
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

bang_nguyen_am= [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                 ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                 ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                 ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                 ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                 ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                 ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                 ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                 ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                 ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                 ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                 ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Chuẩn hóa unicode
# Có 2 loại unicode : unicode tổ hơp và unicode dựng sẵn, điêu này dẫn tới việc 2 từ giống nhau sẽ bị coi là khác nhau
# Chuẩn hóa tất cả về 1 loại là unicode dựng sẵn
def chuan_hoa_unicode(text):
    text = unicodedata.normalize('NFC', text)
    return text

# Có 2 kiểu gõ dấu ở Tiếng Việt, ví dụ như là : òa hoặc oà (ta gọi lần lượt là chuẩn 1 và 2). Mặc dù kiểu gõ chữ sau ít
#phổ biến hơn tuy nhiên vẫn cần phải chuẩn hóa tránh việc một số văn bản vẫn sử dụng kiểu gõ dấu thứ 2.
"""
	Hàm này xử lý chuẩn hóa từng từ một, sau khi chuẩn hóa từng từ thì ta sẽ đi chuân hóa từng câu sau
	"""
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# Tách từ tiếng việt, từ tiếng việt không giống như tiếng anh, tách từ tiếng anh ta chỉ cần tách bằng khoảng trắng
# Tuy nhiên từ tiếng Việt có cả từ đơn lẫn từ ghép nên tách từ tiêng Việt sẽ phúc tạp hơn
# Project sử dung thu viện pyvi (xem mã nguồn tại : https://github.com/trungtv/pyvi) để phục vụ bài toán con tách từ Tiếng Việt
def tach_tu_tieng_viet(text):
    text = ViTokenizer.tokenize(text)
    return text

# Đưa về chữ viết thường
def chuyen_chu_thuong(text):
    return text.lower()

# Xóa đi các dấu cách thừa, các từ không cần thiết cho việc phân loại vẳn bản
def chuan_hoa_cau(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tien_xu_li(text):
    text = chuan_hoa_unicode(text)
    text = chuan_hoa_dau_cau_tieng_viet(text)
    text = tach_tu_tieng_viet(text)
    text = chuyen_chu_thuong(text)
    text = chuan_hoa_cau(text)

    return text

In [1]:

import os

# Set the path to the directory containing the files
path_link = "raw-data/Test_Full/Chinh tri Xa hoi"
Path_final_link="clean-data/Test_Full/Chinh tri Xa hoi"
# Get a list of all files in the directory
file_list = os.listdir(path_link)

# Loop through each file in the list
for file_name in file_list:
    # Check if the file is a regular file (i.e., not a directory)
    if os.path.isfile(os.path.join(path_link, file_name)):
        # Open the file and read its contents
        with open(os.path.join(path_link, file_name), "r", encoding="utf-16") as f:
            file_content = f.read()
            file_content=tien_xu_li(file_content)
        # Save the file content to a new file
        new_file_name = f"{os.path.splitext(file_name)[0]}.txt"
        with open(os.path.join(Path_final_link, new_file_name), "w", encoding="utf-8") as f:
            f.write(file_content)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2936\3687773603.py", line 16, in <module>
    file_content=tien_xu_li(file_content)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2936\4197170719.py", line 155, in tien_xu_li
    text = chuan_hoa_dau_cau_tieng_viet(text)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2936\4197170719.py", line 129, in chuan_hoa_dau_cau_tieng_viet
    cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\regex\regex.py", line 278, in sub
    return pat.sub(repl, string, count, pos, endpos, concurrent, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ADMIN\anaconda3\lib\site

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2936\3687773603.py", line 16, in <module>
    file_content=tien_xu_li(file_content)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2936\4197170719.py", line 155, in tien_xu_li
    text = chuan_hoa_dau_cau_tieng_viet(text)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2936\4197170719.py", line 129, in chuan_hoa_dau_cau_tieng_viet
    cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\regex\regex.py", line 278, in sub
    return pat.sub(repl, string, count, pos, endpos, concurrent, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ADMIN\anaconda3\lib\site

TypeError: object of type 'NoneType' has no len()

In [69]:
import os
from openpyxl import Workbook

# Set the path to the directory containing the files
path_link = "clean-data/Test_Full/Suc khoe"

# Get a list of all files in the directory
file_list = os.listdir(path_link)

# Create a new Excel workbook
wb = Workbook()
ws = wb.active

# Write the header row to the worksheet
ws.append(["Name", "Content"])

# Loop through each file in the list
for file_name in file_list:
    # Check if the file is a regular file (i.e., not a directory)
    if os.path.isfile(os.path.join(path_link, file_name)):
        # Open the file and read its contents
        with open(os.path.join(path_link, file_name), "r", encoding="utf-8") as f:
            file_content = f.read()

        # Write the file name and content to the worksheet
        ws.append(["Sức Khỏe", file_content])

# Save the Excel file
wb.save("Suc_khoe_.xlsx")



In [79]:
import pandas as pd

# Read the first Excel file
df1 = pd.read_excel("Test8.xlsx")

# Read the second Excel file
df2 = pd.read_excel("Van_hoa_.xlsx")

# Concatenate the two dataframes vertically
merged_df = pd.concat([df1, df2], axis=0)

# Save the merged data to a new Excel file
merged_df.to_excel("Full_Finall.xlsx", index=False)


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Read the CSV file into a Pandas dataframe
df = pd.read_csv("csv/Full_Finall.csv")
df = df.replace(np.nan, '', regex=True)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df["Content"], df["Name"], test_size=0.2,random_state=42)

# Convert the text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train a Naive Bayes classifier on the training data
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_counts)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
from sklearn.metrics import f1_score, recall_score

f1 = f1_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred,average='macro')
print("Recall:",recall)
print("F1_Score:",f1)
print("Accuracy:", accuracy)


Recall: 0.8686611384266023
F1_Score: 0.8693233289027098
Accuracy: 0.8916129032258064
